**Study Buddy (PDF Quizzer)**

Goal: Paste a textbook chapter (Context) and ask "What is the definition of X?".

Tech: Extractive QA pipeline.

In [ ]:
!pip install transformers PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.4 MB/s eta 0:00:00


**Import Modules**

In [ ]:
from transformers import pipeline
import PyPDF2


 **Load Pipelines**

In [ ]:
qa_pipeline = pipeline(
    "question-answering",
    model="distilbert-base-cased-distilled-squad"
)

ner_pipeline = pipeline(
    "ner",
    model="dslim/bert-base-NER",
    aggregation_strategy="simple"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 4fd5f9f2-9753-4da1-8a17-14977d7520ff)')' thrown while requesting GET https://huggingface.co/distilbert-base-cased-distilled-squad/resolve/main/tokenizer_config.json
Retrying in 1s [Retry 1/5].


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: cd0dcf75-bad8-4f3d-acb3-98cd83f1c38a)')' thrown while requesting HEAD https://huggingface.co/dslim/bert-base-NER/resolve/main/chat_template.jinja
Retrying in 1s [Retry 1/5].
Device set to use cpu


 **Extract Text from PDF**

In [ ]:
from google.colab import files

uploaded = files.upload()


Saving ai_intro_one_page_filled.pdf to ai_intro_one_page_filled.pdf


In [ ]:
import PyPDF2

# Get the uploaded PDF file name
pdf_filename = list(uploaded.keys())[0]

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
    return text

# Extract text
text = extract_text_from_pdf(pdf_filename)

# Verify extraction
print(text[:500])


Introduction to Artificial Intelligence
 Artificial Intelligence (AI) is a major branch of computer science that focuses on the design and development of
machines capable of performing tasks that normally require human intelligence. Such tasks include learning from
experience, reasoning about complex problems, making informed decisions, understanding natural language,
recognizing patterns, and perceiving the surrounding environment. AI systems are engineered to analyze data,
interpret informatio


**Chunk the Extracted Text**

In [ ]:
def chunk_text(text, chunk_size=400):
    words = text.split()
    chunks = []

    for i in range(0, len(words), chunk_size):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)

    return chunks

chunks = chunk_text(text)
print("Number of chunks:", len(chunks))


Number of chunks: 1


**Load Question Answering Pipeline**

In [ ]:
import os
os.environ["HF_HUB_DOWNLOAD_TIMEOUT"] = "60"


In [ ]:
from transformers import pipeline

qa_pipeline = pipeline(
    "question-answering",
    model="distilbert-base-cased-distilled-squad"
)


Device set to use cpu


**Answer a Question (Core Logic)**

In [ ]:
def answer_question(question, chunks):
    best_answer = {"answer": "", "score": 0}
    best_context = ""

    for chunk in chunks:
        result = qa_pipeline(
            question=question,
            context=chunk,
            max_answer_len=120
        )

        if result["score"] > best_answer["score"]:
            best_answer = result
            best_context = chunk

    return {
        "answer": best_answer["answer"],
        "score": best_answer["score"],
        "context": best_context
    }


 **Ask Your Question**

In [ ]:
questions = [
    "What is Artificial Intelligence?",
    "What is an intelligent agent?",
    "What is a rational agent?",
    "What is meant by thinking like humans in artificial intelligence?",
    "What are the applications of artificial intelligence?"
]

for q in questions:
    result = answer_question(q, chunks)

    print("=" * 80)
    print("Question:")
    print(q)

    print("\nRetrieved Context:")
    print(result["context"][:800])   # limit for readability

    print("\nExtracted Answer:")
    print(result["answer"])

    print("\nConfidence Score:")
    print(result["score"])



Question:
What is Artificial Intelligence?

Retrieved Context:
Introduction to Artificial Intelligence Artificial Intelligence (AI) is a major branch of computer science that focuses on the design and development of machines capable of performing tasks that normally require human intelligence. Such tasks include learning from experience, reasoning about complex problems, making informed decisions, understanding natural language, recognizing patterns, and perceiving the surrounding environment. AI systems are engineered to analyze data, interpret information, and take actions that help achieve predefined objectives efficiently and accurately. One of the foundational concepts in artificial intelligence is the idea of an intelligent agent. An intelligent agent is an entity that perceives its environment through sensors and acts upon that environment usin

Extracted Answer:
a major branch of computer science

Confidence Score:
0.4765491187572479
Question:
What is an intelligent agent?

Ret